## JavaScript函数、闭包、模块

  Javascript中，函数是头等对象，其处理方式与其他JavaScript对象无异，和其他JavaScript数据类型一样，函数可以被变量引用，可以使用字面形式声明，甚至可以作为函数参数传递。

### 函数的字面形式

函数的字面形式由4部分组成:
- function关键字
- 可选的函数名称(合法的JavaScript标识符)
- 放在括号内的参数列表(可以为空)
- 包含在大括号内的函数体(一系列JavaScript语句)

### 函数声明

#### 1. 函数语句

使用函数语句来声明,使用这种方法声明，函数的内容会被编译并创建一个与函数同名的对象。

In [1]:
// 使用函数语句来声明
function add(a,b) {
    return a + b;
}
c = add(1,2);
console.log(c); // 打印出3


3


#### 2. 函数表达式 

使用函数表达式可以创建匿名函数并赋给变量，该变量随后可以调用函数，使用这种方法的问题在于无法递归调用此函数，解决这种限制的方法是采用具名函数表达式。

In [ ]:
// 使用匿名函数表达式声明

var add = function(a,b) {
    return a+b;
};

c = add(1,2);
console.log(c);

In [2]:
// 使用具名函数表达式

var facto = function factorial(n) {
    if (n <= 1) {
        return 1;
    } 
    return n * factorial(n-1);
};

console.log(facto(3));

6


还可以创建调用自身的函数表达式。

In [3]:
(function sayHello() {
    console.log("Hello");
})();

Hello


### 函数作为数据

In [4]:
var validateDataForAge = function(data) {
    person = data();
    console.log(person);
    if (person.age < 1 || person.age >99) {
        return true;
    } else {
        return false;
    }
};

var errorHandlerForAge = function(error) {
    console.log("Error while processing age");
};

function parseRequest(data, validateData, errorHandler) {
    var error = validateData(data);
    if(!error) {
        console.log("no errors");
    } else {
        errorHandler();
    }
}

var generateDataForScientist = function() {
    return {
        name: "Albert Einstein",
        age: Math.floor(Math.random()*(100 - 1)) + 1
    };
}

var generateDataForComposer = function() {
    return {
        name: "JS Bach",
        age: Math.floor(Math.random()*(100 - 1)) + 1
    };
}

parseRequest(generateDataForScientist, validateDataForAge, errorHandlerForAge);
parseRequest(generateDataForComposer, validateDataForAge, errorHandlerForAge);


{ name: 'Albert Einstein', age: 23 }
no errors
{ name: 'JS Bach', age: 59 }
no errors


这里定义了一个通用的parseRequest(),它接受三个函数作为参数，负责将特定的部分组合在一起: 数据、验证器和错误与处理程序。可以对parseRequest函数进行扩展和定制，同时因为每次发起请求都要调用该函数，所以可以作为一个清晰的调试点。

### 作用域

一个变量的作用域就是这个变量的上下文。作用域指明了可以从哪里访问到这个变量，以及在该上下文是否可以访问到这个变量。作用域分为全局作用域和局部作用域。

#### 1. 全局作用域

声明的所有变量都定义在全局作用域，全局变量在所有的作用域都可见，所以在任何作用域都可以修改全局变量。
可以使用两种方式创建全局变量:
- 在所有函数外部使用var声明变量
- 在声明变量时忽略var语句(隐式全局变量)

In [5]:
var a = 1;
function test() {
    a = 2;
    console.log(a);
}

console.log(a);
test();
console.log(a);

1
2
2


#### 2. 局部作用域

和大多数编程语言不同，JavaScript没有块作用域(变量在大括号中),但是有函数作用域,在函数内声明的变量是局部变量，只能由该函数中或是由该函数中的函数使用。

In [6]:
var scope_name = "Global";

function showScopeName() {
    var scope_name = "Local";
    console.log(scope_name);
}
console.log(scope_name);

showScopeName();

Global
Local


JavaScript使用作用域链来建立某个函数的作用域。通常只有一个全局作用域，每个函数有自己的嵌套作用域。在函数内部定义的函数也有局部作用域，该作用域与外围函数的作用域是链接在一起的，函数在作用域中的位置与其出现在源代码中的位置是一致的，在解析时，JavaScript从最内的作用域开始向外搜索。

In [8]:
var testNum = 4;
function testOuter() {
    var testNum = 5;
    function testInner1() {
        console.log(testNum);
    }
    function testInner2() {
        var testNum = 6;
        console.log(testNum);
    }
    testInner1();
    testInner2();
}
testOuter();

5
6


利用函数作为作用域在JavaScript中实现模块化和正确性

In [9]:
var a = 1;
// 引入函数作用域
function foo() {
    var a = 2;
    console.log(a);
}
foo();
console.log(a);

2
1


但这种方式为了创建函数作用域而创建具名函数，却造成了全局作用域或父作用域的污染。另外还必须不停地调用这些函数，由此所引入的大量编写套路会使得代码的可读性越来越差。不过，可以创建能够立即执行的函数(**实际上是个函数表达式,IIFE**)来解决这个问题,它可以避免作用域的污染。

In [10]:
var a = 1;
(function foo() {
    var a = 2;
    console.log(a);
})();  // 该函数立即执行
console.log(a);

2
1


但是以IIFE的形式创建匿名函数还是有一些缺点:
- 在栈调试过程中无法看到函数名，很难对这种代码进行调试
- 无法对匿名函数进行递归
- 过多地使用匿名IIFE有时使代吗难以阅读

IIFE也能传递参数

In [11]:
(function foo(b) {
    var a = 2;
    console.log(a + b);
})(3);

5


行内函数表达式还有一种流行的用法，作为其他函数的参数。

In [13]:
function setActiveTab(activeTabHandler, tab) {
    activeTabHandler();
}

setActiveTab( function () {
    console.log("Setting active tab");
}, 1);

Setting active tab


#### 3. 块作用域
新的ES6引入关键字let，用于生成传统的块作用域。但就目前而言，大多数主流浏览器默认并不支持ES6

In [14]:
var foo = true;

if(foo) {
    let bar = 42;
    console.log(bar);
}

console.log(bar);

42


ReferenceError: bar is not defined

#### 4. 变量提升

变量和函数的声明被移到了代码的顶部(就是常说的变量提升)，只有声明才会被提升，而赋值和其他可执行的逻辑仍然保留在原位置。关于提升，它是以作用域为单位的，在函数中，变量声明会被提升到函数的顶部。

In [13]:
foo();
function foo() {
    console.log(a);
    var a = 1;
    console.log(a);
}

undefined
1


对于函数定义的两种方式，在变量提升方面也有不同的表现

In [18]:
functionTwo();
functionOne();

var functionOne = function() {
    console.log("functionOne");
}

function functionTwo() {
    console.log("functionTwo");
}

functionTwo


TypeError: functionOne is not a function

关于声明，需要注意的是不要根据条件来声明变量或函数，以免产生无法预测的结果。

### arguments参数

arguments参数是传递给函数的所有参数的集合，这个集合有个length属性，包含参数的个数，单个参数的值可以使用索引获得，需要注意的是，arguments并不是JavaScript数组，不要在其上使用数组方法。

In [20]:
var sum = function() {
    var i, total = 0;
    for(i = 0; i < arguments.length; i++) {
        total += arguments[i];
    }
    return total;
}

console.log(sum(1,2,3,4,5,6,7,8,9,10,11,12));

78


### this参数

当函数被调用时，除了在调用时明确给出的哪些参数，还有个叫做this的隐式参数也会传入函数，它指向一个与次函数调用相关联的对象，该对象称为函数上下文。
this的值是由函数的调用上下文以及调用位置决定的:
- 当this用于全局上下文: 如果是在全局上下文中使用this，它就会被绑定到全局上下文。
- 当this用于对象方法中: 在这种情况下, this被赋值或绑定到包含对象上。
- 如果不存在上下文: 当一个函数不跟随任何对象调用时，就不会有上下文，默认情况下，它就会被绑定到全局上下文。如果在这种函数中使用this,它也会被绑定到全局上下文。
- 当this被用于构造函数中: this指向被构造的对象。

#### 1. 作为函数调用

如果函数不是以方法、构造函数或通过apply(),call()调用，那么它只是作为一个函数被调用,当函数以这种方式被调用时，this被绑定到全局对象上。

In [5]:
function add() {}
add();

var substract = function() {
    console.log(this);
};
substract();

Object [global] {
  global: [Circular],
  process:
   process {
     title: '',
     version: 'v10.19.0',
     versions:
      { http_parser: '2.9.3',
        node: '10.19.0',
        v8: '6.8.275.32-node.55',
        uv: '1.34.2',
        zlib: '1.2.11',
        brotli: '1.0.7',
        ares: '1.15.0',
        modules: '64',
        nghttp2: '1.40.0',
        napi: '5',
        openssl: '1.1.1d',
        icu: '66.1',
        unicode: '13.0',
        cldr: '36.1',
        tz: '2022g' },
     arch: 'x64',
     platform: 'linux',
     release:
      { name: 'node',
        lts: 'Dubnium',
        sourceUrl:
         'https://nodejs.org/download/release/v10.19.0/node-v10.19.0.tar.gz',
        headersUrl:
         'https://nodejs.org/download/release/v10.19.0/node-v10.19.0-headers.tar.gz' },
     argv: [ '/usr/bin/node' ],
     execArgv:
      [ '--eval',
        '(function() {\n/*\n * BSD 3-Clause License\n *\n * Copyright (c) 2018, Nicolas Riesco and others as credited in the AUTHORS fil

#### 2. 作为方法调用

方法作为对象属性的函数。对于方法来说，this被绑定在方法被调用时的对象上。

In [1]:
var person = {
    name: "John",
    age: 21,
    greet: function () {
        console.log(this.name);
    }
};
person.greet();

John


#### 3. 作为构造函数调用

构造函数的声明和其他函数一样，但调用时前面要加上`new`关键字。这样的话，this就被绑定到新创建的对象上了。

In [6]:
var Person = function (name) {
    this.name = name;
};
Person.prototype.greet = function () {
    return this.name;
};
var albert = new Person('Albert Einstein');
console.log(albert.greet());

Albert Einstein


#### 4. 通过apply()和call()方法调用

JavaScript中函数也是对象，也具备自己的方法，要想使用apply()方法调用函数，需要传入两个参数: 作为函数上下文的对象 和 作为调用参数的数组;call()方法类似，不过参数不以数组形式传入，而是直接写成参数列表的形式。

### 匿名函数

#### 1. 对象创建过程中的匿名函数

In [7]:
var santa = {
    say: function () {
        console.log("homo");
    }
}
santa.say();

homo


#### 2. 列表创建过程中的匿名函数

In [9]:
var things = [
    function () { console.log("ThingsOne"); },
    function () { console.log("ThingsTwo"); },
];

for (var x=0; x<things.length; x++) {
    things[x]();
}

ThingsOne
ThingsTwo


#### 3. 作为函数参数的匿名函数

In [10]:
function eventHandler(event) {
    event();
}

eventHandler(function() {
    console.log("Event fired");
});

Event fired


#### 4. 出现在条件逻辑中的匿名函数

In [30]:
var shape;

if(shape_name === "SQUARE") {
    shape = function() {
        return "drawing square";
    }
} else {
    shape = function() {
        return "drawing twice";
    }
}

console.log(shape());

drawing twice


### 闭包

闭包是函数声明时创建的作用域，它使得函数能够访问并处理函数的外部变量。换句话说，闭包可以让函数访问到在函数声明时处于作用域内的所有变量以及其他函数。

#### 1. 普通用法

In [ ]:
var outer = 'I am outer';

function outerFn() {
    console.log(outer);
}

outerFn();

In [33]:
var outer = "Outer";
var copy;

function outerFn() {
    var inner = "Inner";
    function innerFn() {
        console.log(outer);
        console.log(inner);
    }
    copy = innerFn;
}
outerFn();
var outer = "Outer2";
copy();

Outer2
Inner


尽管声明innerFn函数的函数作用域已经不存在，但它仍然可以通过闭包访问到当初声明时所在的作用域。

#### 2. 计时器和回调函数

In [34]:
function delay(message) {
    setTimeout(function timeFn() {
        console.log(message);
    }, 1000);
}
delay("Hello, world!");

Hello, world!


#### 3. 私有变量

闭包常用来将信息封装成私有变量的形式。

In [36]:
function privateTest() {
    var points = 0;
    this.getPoints = function() {
        return points;
    };
    this.score = function() {
        points++;
    };
}
var private = new privateTest();
console.log(private.points);
private.score();
console.log(private.getPoints());

undefined
1


通过创建函数作用域内的局部变量，并创建getter和setter函数，我们可以以一种受控的方式更新访问私有变量的效果。

#### 4. 循环和闭包

In [39]:
for(var i=1; i<=5; i++) {
    setTimeout( function delay() {
        console.log(i);
    }, i*100);
}

Timeout {
  _called: false,
  _idleTimeout: 500,
  _idlePrev: [TimersList],
  _idleNext: [TimersList],
  _idleStart: 3270041,
  _onTimeout: [Function: delay],
  _timerArgs: undefined,
  _repeat: null,
  _destroyed: false,
  [Symbol(unrefed)]: false,
  [Symbol(asyncId)]: 279,
  [Symbol(triggerId)]: 268 }

6
6
6
6
6


每个函数访问的闭包中变量的值为最近一次更新的值,可以通过函数作用域来解决这个问题。

In [41]:
for(var i=1; i<=5; i++) {
    (function(j) {
        setTimeout( function delay() {
            console.log(j);
        }, j*100);   
    })(i);
}

1
2
3
4
5


#### 5. 模块

模块可用于模拟类，强调的是对变量和函数的公共和私有访问。模块有助于减少全局作用域的污染。

```javascript
var moduleName = function() {
    // 私有状态
    // 私有函数
    return {
        // 公共状态
        // 公共函数
    }
}
```
要实现以上模式需要实现两点:
- 必须有一个外围函数，至少需要执行一次
- 外围函数必须返回至少一个内部函数，这需要创建一个涵盖了私有状态的闭包，否则无法访问私有状态

In [44]:
var superModule = (function() {
    var secret = "supersecretkey";
    var passcode = "nuke";
    function getSecret() {
        console.log(secret);
    }
    function getPassCode() {
        console.log(passcode);
    }

    return {
        getSecret: getSecret,
        getPassCode: getPassCode
    };
})();
superModule.getSecret();
superModule.getPassCode();

supersecretkey
nuke


### 编码风格

1. 使用函数声明，不使用函数表达式
2. 绝不在非函数块中声明函数
3. 不要给函数参数起arguments的名字